In [ ]:
import graphlab

In [ ]:
mec = graphlab.SFrame('data/sppmecdata.csv')

In [1]:
mec['buysig'] = mec['diffmec'] < 0
mec.head()

NameError: name 'mec' is not defined

In [ ]:
graphlab.canvas.set_target('ipynb')

In [ ]:
trdata,tedata = mec.random_split(.8,seed=0)

In [ ]:
sppfeatures = ['loadforecast','windforecast','natgasprice','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
mec[sppfeatures].show()

In [ ]:
mec.show(view='BoxWhisker Plot', x='hour', y='diffmec')

In [ ]:
spp_regression_model = graphlab.linear_regression.create(trdata,target='diffmec',features=sppfeatures,validation_set=tedata)

In [ ]:
print spp_regression_model.evaluate(tedata)

In [ ]:
tint = {'windforecast':[7500],
        'loadforecast':[28777],
        'natgasprice':[3.20],
        'hour':[8]}

print spp_regression_model.predict(graphlab.SFrame(tint))

In [ ]:


spp_classifier_model = graphlab.logistic_classifier.create(trdata,
                                                     target='buysig',
                                                     features=sppfeatures,
                                                     validation_set=tedata)

In [ ]:
spp_classifier_model.evaluate(tedata, metric='roc_curve')

In [ ]:
spp_classifier_model.show(view='Evaluation')

In [ ]:
mec['predicted_buysig'] = spp_classifier_model.predict(mec, output_type='probability')
reg = spp_regression_model.predict(graphlab.SFrame(tint))
cla = spp_classifier_model.predict(graphlab.SFrame(tint))
print 'Regression model predicts diffmec of ' + str(reg)
print 'Classifier model predicts a DA buy? (0:Sell , 1:Buy) : ' + str(cla)